# Lab 10: Redes Neuronales Recurrentes y LSTM - Práctica

## Objetivos
1. Implementar RNN desde cero
2. Entender la arquitectura LSTM y sus puertas
3. Comparar RNN, LSTM y GRU
4. Aplicar a series temporales
5. Generar secuencias de texto
6. Visualizar hidden states y puertas

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.append('..')
from codigo.rnn_lstm import *

np.random.seed(42)
plt.style.use('default')

print("✓ Librerías importadas correctamente")

## Parte 1: RNN Vanilla - Conceptos Básicos

### 1.1 Crear una RNN Simple

Empezamos con una RNN básica que procesa secuencias.

In [ ]:
# Parámetros
input_size = 1
hidden_size = 8
output_size = 1

# Crear RNN
rnn = RNNNumPy(input_size, hidden_size, output_size, seed=42)

print(f"RNN creada:")
print(f"  Input size: {input_size}")
print(f"  Hidden size: {hidden_size}")
print(f"  Output size: {output_size}")
print(f"\nForma de pesos:")
print(f"  W_xh (input->hidden): {rnn.cell.W_xh.shape}")
print(f"  W_hh (hidden->hidden): {rnn.cell.W_hh.shape}")
print(f"  W_hy (hidden->output): {rnn.W_hy.shape}")

### 1.2 Procesar una Secuencia Simple

Vamos a procesar una secuencia sinusoidal.

In [ ]:
# Generar secuencia de seno
seq_len = 20
t = np.linspace(0, 4 * np.pi, seq_len)
X = np.sin(t).reshape(seq_len, input_size, 1)

print(f"Secuencia de entrada:")
print(f"  Shape: {X.shape}")
print(f"  (seq_len, input_size, batch_size)")

# Forward pass
outputs, hidden_states = rnn.forward(X)

print(f"\nSalidas:")
print(f"  Shape: {outputs.shape}")
print(f"  Hidden states guardados: {len(hidden_states)}")

# Visualizar
fig, axes = plt.subplots(2, 1, figsize=(12, 6))

axes[0].plot(t, X[:, 0, 0], 'b-', label='Input (seno)', linewidth=2)
axes[0].set_ylabel('Valor')
axes[0].set_title('Entrada: Onda Sinusoidal')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

axes[1].plot(t, outputs[:, 0, 0], 'r-', label='Output RNN', linewidth=2)
axes[1].set_xlabel('Tiempo')
axes[1].set_ylabel('Valor')
axes[1].set_title('Salida de RNN (sin entrenar)')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### 1.3 Visualizar Hidden States

Los hidden states contienen la "memoria" de la RNN.

In [ ]:
# Convertir hidden states a array
hidden_array = np.array([h[:, 0] for h in hidden_states])

print(f"Hidden states shape: {hidden_array.shape}")
print(f"(seq_len={seq_len}, hidden_size={hidden_size})")

# Visualizar evolución de hidden states
plt.figure(figsize=(12, 6))
plt.imshow(hidden_array.T, aspect='auto', cmap='coolwarm', interpolation='nearest')
plt.colorbar(label='Activación')
plt.xlabel('Paso Temporal')
plt.ylabel('Neurona Oculta')
plt.title('Evolución de Hidden States en RNN')
plt.tight_layout()
plt.show()

print("\n📊 Cada columna representa el estado de todas las neuronas en un momento.")
print("   Las neuronas aprenden patrones temporales específicos.")

## Parte 2: LSTM - Arquitectura Completa

### 2.1 Crear LSTM y Procesar Secuencia

In [ ]:
# Crear LSTM con misma configuración
lstm = LSTMNumPy(input_size, hidden_size, output_size, seed=42)

print("LSTM creada con 3 puertas:")
print("  1. Forget Gate (f_t): Decide qué olvidar")
print("  2. Input Gate (i_t): Decide qué recordar")
print("  3. Output Gate (o_t): Decide qué exponer\n")

print(f"Pesos de la celda LSTM:")
print(f"  W_f (forget): {lstm.cell.W_f.shape}")
print(f"  W_i (input): {lstm.cell.W_i.shape}")
print(f"  W_C (candidato): {lstm.cell.W_C.shape}")
print(f"  W_o (output): {lstm.cell.W_o.shape}")

# Forward pass
outputs_lstm, hidden_lstm, cell_lstm = lstm.forward(X)

print(f"\nSalidas LSTM:")
print(f"  Outputs: {outputs_lstm.shape}")
print(f"  Hidden states: {len(hidden_lstm)}")
print(f"  Cell states: {len(cell_lstm)}")

### 2.2 Visualizar Puertas LSTM

Veamos cómo las puertas se activan durante el procesamiento.

In [ ]:
# Procesar secuencia y guardar activaciones de puertas
h_t = np.zeros((hidden_size, 1))
C_t = np.zeros((hidden_size, 1))

forget_gates = []
input_gates = []
output_gates = []
cell_states = []

for t in range(seq_len):
    h_t, C_t = lstm.cell.forward(X[t], h_t, C_t)
    forget_gates.append(lstm.cell.cache['f_t'][:, 0])
    input_gates.append(lstm.cell.cache['i_t'][:, 0])
    output_gates.append(lstm.cell.cache['o_t'][:, 0])
    cell_states.append(C_t[:, 0])

# Convertir a arrays
forget_gates = np.array(forget_gates)
input_gates = np.array(input_gates)
output_gates = np.array(output_gates)
cell_states = np.array(cell_states)

# Visualizar las 3 puertas
fig, axes = plt.subplots(4, 1, figsize=(14, 10))

im1 = axes[0].imshow(forget_gates.T, aspect='auto', cmap='RdYlGn', vmin=0, vmax=1)
axes[0].set_title('Forget Gate (f_t): Verde = Recordar, Rojo = Olvidar', fontsize=12)
axes[0].set_ylabel('Neurona')
plt.colorbar(im1, ax=axes[0], label='Activación [0-1]')

im2 = axes[1].imshow(input_gates.T, aspect='auto', cmap='RdYlGn', vmin=0, vmax=1)
axes[1].set_title('Input Gate (i_t): Verde = Aceptar nueva info, Rojo = Rechazar', fontsize=12)
axes[1].set_ylabel('Neurona')
plt.colorbar(im2, ax=axes[1], label='Activación [0-1]')

im3 = axes[2].imshow(output_gates.T, aspect='auto', cmap='RdYlGn', vmin=0, vmax=1)
axes[2].set_title('Output Gate (o_t): Verde = Exponer, Rojo = Ocultar', fontsize=12)
axes[2].set_ylabel('Neurona')
plt.colorbar(im3, ax=axes[2], label='Activación [0-1]')

im4 = axes[3].imshow(cell_states.T, aspect='auto', cmap='coolwarm')
axes[3].set_title('Cell State (C_t): Memoria a largo plazo', fontsize=12)
axes[3].set_xlabel('Paso Temporal')
axes[3].set_ylabel('Neurona')
plt.colorbar(im4, ax=axes[3], label='Valor')

plt.tight_layout()
plt.show()

print("\n📊 Interpretación:")
print("   • Forget Gate (verde): La neurona retiene información")
print("   • Input Gate (verde): La neurona acepta nueva información")
print("   • Output Gate (verde): La neurona expone su estado")
print("   • Cell State: Memoria que persiste a través del tiempo")

### 2.3 Comparar RNN vs LSTM

Visualicemos las diferencias en los hidden states.

In [ ]:
# Comparar hidden states
hidden_rnn = np.array([h[:, 0] for h in hidden_states])
hidden_lstm_array = np.array([h[:, 0] for h in hidden_lstm])

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

im1 = axes[0].imshow(hidden_rnn.T, aspect='auto', cmap='coolwarm')
axes[0].set_title('Hidden States - RNN Vanilla', fontsize=12, fontweight='bold')
axes[0].set_xlabel('Paso Temporal')
axes[0].set_ylabel('Neurona Oculta')
plt.colorbar(im1, ax=axes[0])

im2 = axes[1].imshow(hidden_lstm_array.T, aspect='auto', cmap='coolwarm')
axes[1].set_title('Hidden States - LSTM', fontsize=12, fontweight='bold')
axes[1].set_xlabel('Paso Temporal')
axes[1].set_ylabel('Neurona Oculta')
plt.colorbar(im2, ax=axes[1])

plt.tight_layout()
plt.show()

print("\n🔍 Diferencias:")
print("   • RNN: Estados más volátiles, cambian más bruscamente")
print("   • LSTM: Estados más estables, gracias al cell state")
print("   • LSTM puede mantener información por más tiempo")

## Parte 3: GRU - Alternativa Simplificada

### 3.1 Crear y Comparar GRU

In [ ]:
# Crear GRU
gru = GRUNumPy(input_size, hidden_size, output_size, seed=42)

print("GRU creada con 2 puertas:")
print("  1. Reset Gate (r_t): Controla cuánto pasado usar")
print("  2. Update Gate (z_t): Balance entre pasado y presente\n")

print(f"Pesos de la celda GRU:")
print(f"  W_r (reset): {gru.cell.W_r.shape}")
print(f"  W_z (update): {gru.cell.W_z.shape}")
print(f"  W_h (candidato): {gru.cell.W_h.shape}")

# Forward pass
outputs_gru, hidden_gru = gru.forward(X)

print(f"\nSalidas GRU:")
print(f"  Outputs: {outputs_gru.shape}")
print(f"  Hidden states: {len(hidden_gru)}")

### 3.2 Comparación de Complejidad

Calculemos el número de parámetros de cada arquitectura.

In [ ]:
def count_parameters(input_size, hidden_size, output_size, model_type):
    """Calcular número de parámetros."""
    if model_type == 'RNN':
        # W_xh + W_hh + b_h + W_hy + b_y
        params = (hidden_size * input_size +  # W_xh
                 hidden_size * hidden_size +  # W_hh
                 hidden_size +                # b_h
                 output_size * hidden_size +  # W_hy
                 output_size)                 # b_y
    
    elif model_type == 'LSTM':
        # 4 gates: f, i, C, o
        combined_size = hidden_size + input_size
        params = (4 * (hidden_size * combined_size + hidden_size) +  # 4 gates
                 output_size * hidden_size + output_size)            # output
    
    elif model_type == 'GRU':
        # 3 transformations: r, z, h
        combined_size = hidden_size + input_size
        params = (3 * (hidden_size * combined_size + hidden_size) +  # 3 gates
                 output_size * hidden_size + output_size)            # output
    
    return params

# Calcular parámetros
params_rnn = count_parameters(input_size, hidden_size, output_size, 'RNN')
params_lstm = count_parameters(input_size, hidden_size, output_size, 'LSTM')
params_gru = count_parameters(input_size, hidden_size, output_size, 'GRU')

print("\n" + "="*60)
print("COMPARACIÓN DE ARQUITECTURAS")
print("="*60)
print(f"\n{'Modelo':<15} {'Parámetros':<15} {'Ratio vs RNN':<15}")
print("-"*60)
print(f"{'RNN':<15} {params_rnn:<15} {1.0:<15.2f}")
print(f"{'LSTM':<15} {params_lstm:<15} {params_lstm/params_rnn:<15.2f}")
print(f"{'GRU':<15} {params_gru:<15} {params_gru/params_rnn:<15.2f}")
print("="*60)

# Visualizar
models = ['RNN', 'LSTM', 'GRU']
params = [params_rnn, params_lstm, params_gru]
colors = ['#3498db', '#e74c3c', '#2ecc71']

plt.figure(figsize=(10, 6))
bars = plt.bar(models, params, color=colors, alpha=0.7, edgecolor='black', linewidth=2)

# Añadir valores sobre las barras
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height,
            f'{int(height)}',
            ha='center', va='bottom', fontsize=12, fontweight='bold')

plt.ylabel('Número de Parámetros', fontsize=12)
plt.title(f'Comparación de Complejidad\n(input={input_size}, hidden={hidden_size}, output={output_size})',
         fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.show()

print("\n💡 Conclusiones:")
print(f"   • LSTM tiene ~{params_lstm/params_rnn:.1f}x más parámetros que RNN")
print(f"   • GRU tiene ~{params_gru/params_rnn:.1f}x más parámetros que RNN")
print(f"   • GRU es ~{params_lstm/params_gru:.0f}% más ligero que LSTM")
print("   • Más parámetros = Mayor capacidad pero más lento")

## Parte 4: Series Temporales - Predicción

### 4.1 Generar Serie Temporal Sintética

In [ ]:
# Generar serie temporal compleja
n_points = 200
t = np.linspace(0, 10 * np.pi, n_points)

# Combinar múltiples componentes
trend = 0.05 * t  # Tendencia lineal
seasonal = np.sin(t) + 0.5 * np.sin(2 * t)  # Componentes estacionales
noise = np.random.normal(0, 0.1, n_points)  # Ruido

time_series = trend + seasonal + noise

# Visualizar
plt.figure(figsize=(14, 5))
plt.plot(t, time_series, 'b-', linewidth=1.5, label='Serie Temporal')
plt.plot(t, trend, 'r--', linewidth=2, label='Tendencia', alpha=0.7)
plt.plot(t, seasonal, 'g--', linewidth=2, label='Estacionalidad', alpha=0.7)
plt.xlabel('Tiempo', fontsize=12)
plt.ylabel('Valor', fontsize=12)
plt.title('Serie Temporal Sintética (Tendencia + Estacionalidad + Ruido)', 
         fontsize=14, fontweight='bold')
plt.legend(fontsize=10)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"Serie temporal generada: {len(time_series)} puntos")

### 4.2 Preparar Datos para Entrenamiento

Dividimos la serie en secuencias de entrada/salida.

In [ ]:
# Normalizar datos
data_mean = time_series.mean()
data_std = time_series.std()
time_series_norm = (time_series - data_mean) / data_std

# Crear secuencias
seq_length = 10  # Usar 10 pasos para predecir el siguiente
pred_length = 1  # Predecir 1 paso adelante

X_seq, y_seq = create_sequences(time_series_norm, seq_length, pred_length)

print(f"Secuencias creadas:")
print(f"  X shape: {X_seq.shape} (num_sequences, seq_length)")
print(f"  y shape: {y_seq.shape} (num_sequences, pred_length)")

# Dividir en train/test
split = int(0.8 * len(X_seq))
X_train, X_test = X_seq[:split], X_seq[split:]
y_train, y_test = y_seq[:split], y_seq[split:]

print(f"\nDivisión train/test:")
print(f"  Train: {len(X_train)} secuencias")
print(f"  Test:  {len(X_test)} secuencias")

# Mostrar ejemplo de secuencia
print(f"\nEjemplo de secuencia:")
print(f"  Input (10 pasos):  {X_train[0]}")
print(f"  Target (1 paso):   {y_train[0]}")

### 4.3 Entrenar con PyTorch (si está disponible)

In [ ]:
if PYTORCH_AVAILABLE:
    import torch
    import torch.nn as nn
    import torch.optim as optim
    from torch.utils.data import DataLoader, TensorDataset
    
    # Preparar datos para PyTorch
    X_train_t = torch.FloatTensor(X_train).unsqueeze(-1)  # (batch, seq, 1)
    y_train_t = torch.FloatTensor(y_train).squeeze()
    X_test_t = torch.FloatTensor(X_test).unsqueeze(-1)
    y_test_t = torch.FloatTensor(y_test).squeeze()
    
    # Transponer para (seq, batch, features)
    X_train_t = X_train_t.transpose(0, 1)
    X_test_t = X_test_t.transpose(0, 1)
    
    print(f"Datos preparados para PyTorch:")
    print(f"  X_train: {X_train_t.shape} (seq, batch, features)")
    print(f"  y_train: {y_train_t.shape} (batch,)")
    
    # Crear modelo
    model = SimpleLSTM(
        input_size=1,
        hidden_size=32,
        output_size=1,
        num_layers=1,
        bidirectional=False
    )
    
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.01)
    
    print(f"\nModelo LSTM creado:")
    print(model)
    
    # Entrenar
    num_epochs = 50
    losses = []
    
    model.train()
    for epoch in range(num_epochs):
        optimizer.zero_grad()
        output, _ = model(X_train_t)
        loss = criterion(output.squeeze(), y_train_t)
        loss.backward()
        optimizer.step()
        losses.append(loss.item())
        
        if (epoch + 1) % 10 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.6f}')
    
    # Visualizar curva de aprendizaje
    plt.figure(figsize=(10, 5))
    plt.plot(losses, linewidth=2)
    plt.xlabel('Época', fontsize=12)
    plt.ylabel('MSE Loss', fontsize=12)
    plt.title('Curva de Aprendizaje - LSTM', fontsize=14, fontweight='bold')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    # Evaluar
    model.eval()
    with torch.no_grad():
        train_pred, _ = model(X_train_t)
        test_pred, _ = model(X_test_t)
    
    train_pred = train_pred.squeeze().numpy()
    test_pred = test_pred.squeeze().numpy()
    
    # Visualizar predicciones
    plt.figure(figsize=(14, 5))
    
    # Train
    plt.subplot(1, 2, 1)
    plt.scatter(y_train, train_pred, alpha=0.5, s=20)
    plt.plot([y_train.min(), y_train.max()], 
            [y_train.min(), y_train.max()], 'r--', linewidth=2)
    plt.xlabel('Valor Real', fontsize=11)
    plt.ylabel('Predicción', fontsize=11)
    plt.title('Train Set', fontsize=12, fontweight='bold')
    plt.grid(True, alpha=0.3)
    
    # Test
    plt.subplot(1, 2, 2)
    plt.scatter(y_test, test_pred, alpha=0.5, s=20, color='orange')
    plt.plot([y_test.min(), y_test.max()], 
            [y_test.min(), y_test.max()], 'r--', linewidth=2)
    plt.xlabel('Valor Real', fontsize=11)
    plt.ylabel('Predicción', fontsize=11)
    plt.title('Test Set', fontsize=12, fontweight='bold')
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Calcular métricas
    train_mse = np.mean((y_train - train_pred) ** 2)
    test_mse = np.mean((y_test - test_pred) ** 2)
    
    print(f"\n📊 Resultados:")
    print(f"   Train MSE: {train_mse:.6f}")
    print(f"   Test MSE:  {test_mse:.6f}")
    print(f"   ✓ Modelo entrenado exitosamente!")
    
else:
    print("⚠️ PyTorch no disponible. Saltando entrenamiento.")

## Parte 5: Generación de Texto Simple

### 5.1 Crear Vocabulario Simple

In [ ]:
# Texto simple para demostración
text = "hola mundo hola python hola rnn lstm gru deep learning"

# Crear vocabulario
words = text.split()
vocab = sorted(set(words))
word_to_idx = {word: idx for idx, word in enumerate(vocab)}
idx_to_word = {idx: word for word, idx in word_to_idx.items()}

print(f"Texto: '{text}'")
print(f"\nVocabulario ({len(vocab)} palabras):")
for word, idx in word_to_idx.items():
    print(f"  {word:12} -> {idx}")

# Convertir a índices
text_indices = [word_to_idx[word] for word in words]
print(f"\nTexto como índices: {text_indices}")

### 5.2 Modelo Char-Level (Caracteres)

Un modelo más interesante: predecir el siguiente carácter.

In [ ]:
if PYTORCH_AVAILABLE:
    # Texto más largo para char-level
    text_char = """El deep learning es fascinante. 
    Las redes neuronales recurrentes pueden aprender patrones en secuencias. 
    LSTM y GRU son arquitecturas poderosas."""
    
    # Crear vocabulario de caracteres
    chars = sorted(set(text_char))
    char_to_idx = {ch: idx for idx, ch in enumerate(chars)}
    idx_to_char = {idx: ch for ch, idx in char_to_idx.items()}
    vocab_size = len(chars)
    
    print(f"Texto length: {len(text_char)} caracteres")
    print(f"Vocabulario: {vocab_size} caracteres únicos")
    print(f"Caracteres: {''.join(chars)}")
    
    # Convertir texto a índices
    text_encoded = [char_to_idx[ch] for ch in text_char]
    
    # Crear secuencias
    seq_len_char = 20
    X_char, y_char = [], []
    
    for i in range(len(text_encoded) - seq_len_char):
        X_char.append(text_encoded[i:i+seq_len_char])
        y_char.append(text_encoded[i+seq_len_char])
    
    X_char = torch.LongTensor(X_char)  # (batch, seq_len)
    y_char = torch.LongTensor(y_char)  # (batch,)
    
    print(f"\nSecuencias creadas: {len(X_char)}")
    print(f"Ejemplo:")
    print(f"  Input:  '{text_char[0:seq_len_char]}'")
    print(f"  Target: '{text_char[seq_len_char]}'")
    
    # Crear modelo
    char_model = CharRNN(
        vocab_size=vocab_size,
        embed_size=32,
        hidden_size=64,
        num_layers=1,
        rnn_type='lstm'
    )
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(char_model.parameters(), lr=0.005)
    
    print(f"\nModelo CharRNN creado:")
    print(f"  Vocab size: {vocab_size}")
    print(f"  Embed size: 32")
    print(f"  Hidden size: 64")
    
    # Entrenar
    num_epochs = 100
    batch_size = 32
    
    dataset = TensorDataset(X_char, y_char)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    
    char_losses = []
    char_model.train()
    
    for epoch in range(num_epochs):
        epoch_loss = 0
        for batch_x, batch_y in dataloader:
            optimizer.zero_grad()
            output, _ = char_model(batch_x)
            # Tomar último output
            output_last = output[:, -1, :]  # (batch, vocab_size)
            loss = criterion(output_last, batch_y)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(char_model.parameters(), 5.0)
            optimizer.step()
            epoch_loss += loss.item()
        
        avg_loss = epoch_loss / len(dataloader)
        char_losses.append(avg_loss)
        
        if (epoch + 1) % 20 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}')
    
    plt.figure(figsize=(10, 5))
    plt.plot(char_losses, linewidth=2, color='purple')
    plt.xlabel('Época', fontsize=12)
    plt.ylabel('Cross Entropy Loss', fontsize=12)
    plt.title('Entrenamiento CharRNN', fontsize=14, fontweight='bold')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    print("\n✓ Modelo CharRNN entrenado!")
    
else:
    print("⚠️ PyTorch no disponible.")

### 5.3 Generar Texto

In [ ]:
if PYTORCH_AVAILABLE:
    def generate_text(model, start_text, length=100, temperature=1.0):
        """Generar texto carácter por carácter."""
        model.eval()
        
        # Inicializar con texto de inicio
        current_text = start_text
        input_seq = [char_to_idx.get(ch, 0) for ch in start_text[-seq_len_char:]]
        
        with torch.no_grad():
            for _ in range(length):
                # Preparar input
                x = torch.LongTensor([input_seq]).to('cpu')
                
                # Predecir
                output, _ = model(x)
                logits = output[0, -1, :] / temperature
                probs = torch.softmax(logits, dim=0)
                
                # Samplear siguiente carácter
                next_idx = torch.multinomial(probs, 1).item()
                next_char = idx_to_char[next_idx]
                
                # Actualizar
                current_text += next_char
                input_seq = input_seq[1:] + [next_idx]
        
        return current_text
    
    # Generar con diferentes temperaturas
    start = "El deep learning"
    
    print("\n" + "="*70)
    print("GENERACIÓN DE TEXTO")
    print("="*70)
    
    for temp in [0.5, 1.0, 1.5]:
        generated = generate_text(char_model, start, length=100, temperature=temp)
        print(f"\nTemperatura {temp}:")
        print(f"{generated}")
        print("-"*70)
    
    print("\n💡 Temperatura:")
    print("   • Baja (0.5): Más determinista, repite patrones")
    print("   • Media (1.0): Balance entre creatividad y coherencia")
    print("   • Alta (1.5): Más aleatorio, más creativo pero menos coherente")
    
else:
    print("⚠️ PyTorch no disponible.")

## Parte 6: Ejercicios Prácticos

### Ejercicio 1: Implementar Backward Pass para RNN

Completa la implementación del backward pass.

In [ ]:
# EJERCICIO: Implementa backpropagation through time para RNN

def bptt_simple(X, y_true, rnn_cell, learning_rate=0.01):
    """
    Backpropagation Through Time simple.
    
    Pasos:
    1. Forward pass guardando todos los estados
    2. Calcular gradiente de la pérdida
    3. Backward pass acumulando gradientes
    4. Actualizar pesos
    """
    seq_len = X.shape[0]
    
    # TODO: Implementar BPTT
    # 1. Forward pass
    # 2. Calcular loss
    # 3. Backward pass
    # 4. Actualizar pesos con gradient clipping
    
    pass

print("💪 Ejercicio 1: Implementa BPTT arriba")
print("   Pista: Usa gradient clipping para estabilidad")

### Ejercicio 2: Comparar RNN/LSTM/GRU en una Tarea

Compara los tres modelos en predicción de series temporales.

In [ ]:
# EJERCICIO: Compara RNN, LSTM y GRU

if PYTORCH_AVAILABLE:
    print("💪 Ejercicio 2:")
    print("   1. Crea 3 modelos (RNN, LSTM, GRU) con PyTorch")
    print("   2. Entrena cada uno en la serie temporal")
    print("   3. Compara MSE de test")
    print("   4. Compara tiempo de entrenamiento")
    print("   5. Visualiza resultados")
    
    # TODO: Tu código aquí
    
else:
    print("⚠️ PyTorch no disponible para este ejercicio")

### Ejercicio 3: Bidirectional LSTM

Implementa y prueba un LSTM bidireccional.

In [ ]:
# EJERCICIO: LSTM Bidireccional

if PYTORCH_AVAILABLE:
    print("💪 Ejercicio 3:")
    print("   1. Crea SimpleLSTM con bidirectional=True")
    print("   2. Entrena en clasificación (ejemplo: sentimiento)")
    print("   3. Compara con LSTM unidireccional")
    print("   4. Analiza cuándo bidireccional es mejor")
    
    # TODO: Tu código aquí
    # Pista: Bidireccional duplica el tamaño de hidden state
    
else:
    print("⚠️ PyTorch no disponible para este ejercicio")

## Parte 7: Resumen y Conclusiones

### 7.1 Conceptos Clave Aprendidos

In [ ]:
print("="*70)
print("RESUMEN: Redes Neuronales Recurrentes y LSTM")
print("="*70)

print("\n1. ARQUITECTURAS:")
print("   • RNN Vanilla: Simple, problemas con gradiente")
print("   • LSTM: 3 puertas, resuelve gradiente desvaneciente")
print("   • GRU: 2 puertas, más eficiente que LSTM")

print("\n2. COMPONENTES LSTM:")
print("   • Forget Gate: Controla qué olvidar")
print("   • Input Gate: Controla qué recordar")
print("   • Output Gate: Controla qué exponer")
print("   • Cell State: Memoria a largo plazo")

print("\n3. APLICACIONES:")
print("   • Series Temporales: Predicción, forecasting")
print("   • NLP: Clasificación, generación de texto")
print("   • Secuencias: Cualquier dato con orden temporal")

print("\n4. TÉCNICAS IMPORTANTES:")
print("   • Gradient Clipping: Prevenir explosión")
print("   • Teacher Forcing: Acelerar entrenamiento")
print("   • Bidirectional: Usar contexto futuro")
print("   • Stacked: Múltiples capas para abstracción")

print("\n5. LIMITACIONES:")
print("   • Procesamiento secuencial (no paralelizable)")
print("   • Lento comparado con Transformers")
print("   • Límite en dependencias muy largas")

print("\n" + "="*70)
print("✓ Lab 10 completado!")
print("  Siguiente: Lab 11 - Transformers")
print("="*70)

### 7.2 Recursos Adicionales

**Papers Importantes:**
- [LSTM Original (1997)](http://www.bioinf.jku.at/publications/older/2604.pdf) - Hochreiter & Schmidhuber
- [GRU (2014)](https://arxiv.org/abs/1406.1078) - Cho et al.
- [Understanding LSTM Networks](http://colah.github.io/posts/2015-08-Understanding-LSTMs/) - Chris Olah

**Recursos Online:**
- PyTorch RNN Tutorial: https://pytorch.org/tutorials/intermediate/char_rnn_classification_tutorial.html
- The Unreasonable Effectiveness of RNNs: http://karpathy.github.io/2015/05/21/rnn-effectiveness/

**Datasets para Práctica:**
- Time Series: Stock prices, weather data
- Text: Penn Treebank, WikiText-2, Shakespeare
- Sequences: Human activity, sensor data